In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px

2023-12-11 12:20:13.380 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
df = pd.read_excel('data.xlsx')
df.head(2)

,Supplier Name,Product Line Number,Product Line Description,Buying Group Number,Buying Group Name,Customer #,Parent Customer,Customer Name,Customer Shipping City,Customer Shipping State,...,Channel Number,Channel Description,Segment Code,Segment Description,Tier Number,Tier Description,Product Line Sub-Category,Dot DC,Table,Segment Description 2
0,AWAKE CHOCOLATE,1769,AWAKE CHOCOLATE,11.0,UNIPRO REPORTING,141740-1,J. POLEP,J. POLEP,CHICOPEE,MA,...,4.0,RETAIL,4004.0,CONVENIENCE STORE,1.0,TIER I,GROCERY,LIVERPOOL,Dot,CONVENIENCE STORE
1,AWAKE CHOCOLATE,1769,AWAKE CHOCOLATE,11.0,UNIPRO REPORTING,141740-1,J. POLEP,J. POLEP,CHICOPEE,MA,...,4.0,RETAIL,4004.0,CONVENIENCE STORE,1.0,TIER I,GROCERY,LIVERPOOL,Dot,CONVENIENCE STORE


In [3]:
year = df['Invoice Date'].dt.year.unique()
segment = df['Segment Description 2'].unique()

In [4]:
df_selection = df[(df['Invoice Date'].dt.year.isin(year)) & (df['Segment Description 2'].isin(segment))]

In [5]:
seg_sales = df_selection.groupby('Segment Description 2',as_index=False)['Dollars'].sum()

In [15]:
df.columns

Index(['Supplier Name', 'Product Line Number', 'Product Line Description',
       'Buying Group Number', 'Buying Group Name', 'Customer #',
       'Parent Customer', 'Customer Name', 'Customer Shipping City',
       'Customer Shipping State', 'Customer Shipping Zip Code',
       'Customer Invoice Number', 'Invoice Date', 'Customer PO Number',
       'Customer Order Number', 'Dot #', 'MFG #', 'Item UPC',
       'Item Full Description', 'Qty Ordered', 'Qty Received', 'Dollars',
       'Customer Extended Gross Weight', 'Customer Extended Net Weight',
       'Channel Number', 'Channel Description', 'Segment Code',
       'Segment Description', 'Tier Number', 'Tier Description',
       'Product Line Sub-Category', 'Dot DC', 'Table',
       'Segment Description 2'],
      dtype='object')

In [17]:
df_selection = df[
    (df['Invoice Date'].dt.year.isin(year)) & (df['Segment Description 2'].isin(segment))
    ][['Table',
       'Invoice Date',
       'Parent Customer',
       'Customer Name',
       'MFG #',
       'Item Full Description',
       'Qty Ordered',
       'Qty Received',
       'Dollars',
       'Segment Description 2']].groupby('Parent Customer')

df_selection

,Table,Invoice Date,Parent Customer,Customer Name,MFG #,Item Full Description,Qty Ordered,Qty Received,Dollars,Segment Description 2
0,Dot,2022-01-03,J. POLEP,J. POLEP,00453U,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,8,8.0,1046.40,CONVENIENCE STORE
1,Dot,2022-01-03,J. POLEP,J. POLEP,00460U,CAFFEINATED CHOCOLATE BITES SINGLES PB CHOC,8,8.0,1046.40,CONVENIENCE STORE
2,Dot,2022-01-03,R.C. TAYLOR,R.C. TAYLOR,00105U,AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...,1,1.0,78.48,CONVENIENCE STORE
3,Dot,2022-01-03,Tropical,TROPICAL FOODS--TX,00150U,CAFFEINATED CHOCOLATE BITES SINGLES,2,2.0,261.60,ALTERNATE RETAIL
4,Dot,2022-01-03,Tropical,TROPICAL FOODS--TX,00453U,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,2,2.0,261.60,ALTERNATE RETAIL
...,...,...,...,...,...,...,...,...,...,...
16653,Dot,2023-12-08,Vistar,VISTAR-NEW ENGLAND,00254U,AWAKE CHOCOLATE AWAKE CAFF DARK CHOC- 6X12PK M...,5,5.0,427.70,VENDING
16654,Dot,2023-12-08,Vistar,VISTAR-NEW ENGLAND,00493U,AWAKE NO SUGAR ADDED 6 X 50CT BITES CHANGEMAKE...,3,3.0,423.00,VENDING
16655,Dot,2023-12-08,Vistar,VISTAR-NEW ENGLAND,00494U,AWAKE NO SUGAR ADDED 6 X 50CT BITES CHANGEMAKE...,2,2.0,282.00,VENDING
16656,Dot,2023-12-08,Vistar,VISTAR-PHOENIX,00254U,AWAKE CHOCOLATE AWAKE CAFF DARK CHOC- 6X12PK M...,2,2.0,171.08,VENDING


In [11]:
px.bar(
    seg_sales.sort_values(by = 'Dollars',ascending=False),
    x='Dollars',
    y='Segment Description 2',
    orientation = 'h',
    title = "<b>Sales by Market Segment</b>",
    template = 'plotly_white',
    color='Segment Description 2',
    labels={'Segment Description 2':'Market Segment',
            'Dollars':'Sales in $USD'},
    width=800
).update_layout(showlegend=False)

In [ ]:
year = df['Invoice Date'].dt.year.unique()
segment = df['Segment Description 2'].unique()


df_selection = df[(df['Invoice Date'].dt.year.isin(year)) & (df['Segment Description 2'].isin(segment))]
# df_selection = df[(df['Invoice Date'].dt.year==2023) & (df['Segment Description 2'].isin(segment))]


seg_sales = df_selection.groupby('Segment Description 2',as_index=False)['Dollars'].sum()
fig_seg_sales = px.bar(
    seg_sales.sort_values(by = 'Dollars',ascending=False),
    x='Dollars',
    y='Segment Description 2',
    orientation = 'h',
    title = "<b>Sales by Market Segment</b>",
    template = 'plotly_white',
    color='Segment Description 2',
    labels={'Segment Description 2':'Market Segment',
            'Dollars':'Sales in $USD'}
)
fig_seg_sales.show()

In [ ]:
px.bar(
    seg_sales.sort_values(by = 'Dollars',ascending=False),
    x='Dollars',
    y='Segment Description 2',
    orientation = 'h',
    title = "<b>Sales by Market Segment</b>",
    template = 'plotly_white',
    color='Segment Description 2',
    labels={'Segment Description 2':'Market Segment',
            'Dollars':'Sales in $USD'}
)